In [ ]:
# install datasets
!pip install -qq datasets
!pip install git+https://github.com/vasudevgupta7/transformers.git@add_big_bird

import datasets
import torch

  Cloning https://github.com/vasudevgupta7/transformers.git (to revision add_big_bird) to /tmp/pip-req-build-sudkxurt
  Running command git clone -q https://github.com/vasudevgupta7/transformers.git /tmp/pip-req-build-sudkxurt
  Running command git checkout -b add_big_bird --track origin/add_big_bird
  Switched to a new branch 'add_big_bird'
  Branch 'add_big_bird' set up to track remote branch 'add_big_bird' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
validation_dataset = datasets.load_dataset("trivia_qa", "rc", split="validation")

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae. Subsequent calls will reuse this data.


In [ ]:
validation_dataset

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
    num_rows: 17944
})

In [ ]:
# check out schema
validation_dataset.info.features

{'answer': {'aliases': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'matched_wiki_entity_name': Value(dtype='string', id=None),
  'normalized_aliases': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'normalized_matched_wiki_entity_name': Value(dtype='string', id=None),
  'normalized_value': Value(dtype='string', id=None),
  'type': Value(dtype='string', id=None),
  'value': Value(dtype='string', id=None)},
 'entity_pages': Sequence(feature={'doc_source': Value(dtype='string', id=None), 'filename': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'wiki_context': Value(dtype='string', id=None)}, length=-1, id=None),
 'question': Value(dtype='string', id=None),
 'question_id': Value(dtype='string', id=None),
 'question_source': Value(dtype='string', id=None),
 'search_results': Sequence(feature={'description': Value(dtype='string', id=None), 'filename': Value(dtype='string', id=None), 'rank': Value(dtype='int32', 

In [ ]:
# define the mapping function
def format_dataset(example):
    # the context might be comprised of multiple contexts => me merge them here
    example["context"] = " ".join(("\n".join(example["entity_pages"]["wiki_context"])).split("\n"))
    example["targets"] = example["answer"]["aliases"]
    example["norm_target"] = example["answer"]["normalized_value"]
    return example

# map the dataset and throw out all unnecessary columns
validation_dataset = validation_dataset.map(format_dataset, remove_columns=["search_results", "question_source", "entity_pages", "answer", "question_id"])

  0%|          | 0/17944 [00:00<?, ?ex/s]

In [ ]:
validation_dataset[8]

{'context': '',
 'norm_target': 'nikkei',
 'question': 'What is the Japanese share index called?',
 'targets': ['Nikkei', 'Nikkei (disambiguation)']}

In [ ]:
validation_dataset = validation_dataset.filter(lambda x: len(x["context"]) > 0)
# check out how many samples are left
validation_dataset

In [ ]:
print("\n\nLength for each example")
print(30 * "=")

# length for each example
validation_dataset.map(lambda x, i: print(f"Id: {i} - Question Length: {len(x['question'])} - context Length: {len(x['context'])}"), with_indices=True)
print(30 * "=")

print("\n")
print("Num examples larger than 4 * 3000 characters: ")
# filter out examples smaller than 4 * 3000
short_validation_dataset = validation_dataset.filter(lambda x: (len(x['question']) + len(x['context'])) < 4 * 3000)
short_validation_dataset



Length for each example


  0%|          | 0/16137 [00:00<?, ?ex/s]

Streaming output truncated to the last 5000 lines.
Id: 11141 - Question Length: 46 - context Length: 16006
Id: 11142 - Question Length: 45 - context Length: 12879
Id: 11143 - Question Length: 41 - context Length: 31880
Id: 11144 - Question Length: 88 - context Length: 49771
Id: 11145 - Question Length: 56 - context Length: 100626
Id: 11146 - Question Length: 92 - context Length: 17434
Id: 11147 - Question Length: 59 - context Length: 17095
Id: 11148 - Question Length: 87 - context Length: 119036
Id: 11149 - Question Length: 30 - context Length: 12369
Id: 11150 - Question Length: 88 - context Length: 34938
Id: 11151 - Question Length: 128 - context Length: 110935
Id: 11152 - Question Length: 76 - context Length: 69603
Id: 11153 - Question Length: 176 - context Length: 104830
Id: 11154 - Question Length: 98 - context Length: 73733
Id: 11155 - Question Length: 152 - context Length: 182575
Id: 11156 - Question Length: 189 - context Length: 275765
Id: 11157 - Question Length: 125 - context 

  0%|          | 0/17 [00:00<?, ?ba/s]

Dataset({
    features: ['question', 'context', 'targets', 'norm_target'],
    num_rows: 3084
})

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import BigBirdTokenizer, BigBirdForQuestionAnswering

device = "cpu"
tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-base-trivia-itc")

# download the 1.7 GB model. It might take ~1min
model = BigBirdForQuestionAnswering.from_pretrained("google/bigbird-base-trivia-itc").to(device)

In [ ]:
model.config.block_size

In [ ]:
def alias_answer(answer, include=None):
  alias = answer.replace('_', ' ').lower()
  exclude = set(''.join(['‘', '’', '´', '`', '.', ',', '-']))
  include = include or []
  alias = ''.join(c if c not in exclude or c in include else ' ' for c in alias)
  return ' '.join(alias.split()).strip()


def make_answer_set(given_answers):
  """Apply less aggressive normalization to the answer aliases."""
  answers = []
  for alias in given_answers:
    answers.append(alias_answer(alias))
    answers.append(alias_answer(alias, [',', '.']))
    answers.append(alias_answer(alias, ['-']))
    answers.append(alias_answer(alias, [',', '.', '-']))
  return set(answers)

In [ ]:
def get_best_start_end_idx(start_scores, end_scores, top_k=1, max_size=16):
    # top_k = 1 corresponds to argmax
    best_start_scores, best_start_idx = torch.topk(start_scores, top_k)
    best_end_scores, best_end_idx = torch.topk(end_scores, top_k)

    widths = best_end_idx[:, None] - best_start_idx[None, :]
    mask = torch.logical_or(widths < 0, widths > max_size)
    scores = (best_end_scores[:, None] + best_start_scores[None, :]) - (1e8 * mask)
    best_score = torch.argmax(scores).item()

    return best_start_idx[best_score % top_k], best_end_idx[best_score // top_k]

In [ ]:
def evaluate(example):
    def get_answer(question, context):
        # encode question and context so that they are seperated by a tokenizer.sep_token and cut at max_length
        encoding = tokenizer(question, context, return_tensors="pt", max_length=300, padding="max_length", truncation=True)
        input_ids = encoding.input_ids.to(device)
        attention_mask = encoding.attention_mask.to(device)

        # The scores for the possible start token and end token of the answer are retrived
        # wrap the function in torch.no_grad() to save memory
        with torch.no_grad():
            start_scores, end_scores = model(input_ids=input_ids, attention_mask=attention_mask).to_tuple()

        start_score, end_score = get_best_start_end_idx(start_scores[0], end_scores[0], top_k=8)
        # start_score = torch.argmax(start_scores)
        # end_score = torch.argmax(end_scores)

        # Let's take the most likely token using `argmax` and retrieve the answer

        all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

        answer_tokens = all_tokens[start_score: end_score + 1]
        answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens)).replace('"', '')  # remove space prepending space token and remove unnecessary '"'
        
        return answer

    # save the model's output here
    example["output"] = get_answer(example["question"], example["context"])
    targets = example["targets"]

    targets = targets + [" ".join(x.split(" ")[1:]) for x in targets if len(x.split(" ")) > 1] + [" ".join(x.split(" ")[:-1]) for x in targets if len(x.split(" ")) > 1]

    answers = make_answer_set(targets)
    predictions = make_answer_set([example["output"]])

    # save if it's a match or not
    example["match"] = len(list(answers & predictions)) > 0

    return example

In [ ]:
results_short = short_validation_dataset.map(evaluate)

  0%|          | 0/3084 [00:00<?, ?ex/s]

Attention type 'block_sparse' is not possible if sequence_length: 300 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...
